In [ ]:
import speech_recognition as sr
import os
import cv2
import numpy as np

# İşaret dili videolarının bulunduğu klasör
VIDEO_FOLDER = "İşaretDiliVideo"

# Konuşmayı Algılama Fonksiyonu
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Konuşun...")
        recognizer.adjust_for_ambient_noise(source, duration=1)  # Gürültü ayarı
        audio = recognizer.listen(source)

        try:
            text = recognizer.recognize_google(audio, language="tr-TR")
            print(f"Saptanan Cümle: {text}")
            return text.lower()  # Küçük harfe çevirerek dosya adlarıyla eşleşmesini sağlıyoruz
        except sr.UnknownValueError:
            print("Ses algılanamadı.")
            return None
        except sr.RequestError:
            print("Google API hatası.")
            return None

# Cümledeki kelimeleri videolarla eşleştirme
def find_video_files(sentence):
    words = sentence.split()  # Cümleyi kelimelere ayır
    video_paths = []
    
    for word in words:
        for file in os.listdir(VIDEO_FOLDER):
            if file.lower().startswith(word) and file.endswith((".mp4", ".avi", ".mov")):
                print(f"Video bulundu: {file}")
                video_paths.append(os.path.join(VIDEO_FOLDER, file))
                break  # İlk eşleşmeyi bulduğunda devam et
    
    return video_paths

# Videoları birleştirerek oynatma
def play_videos(video_paths):
    if not video_paths:
        print("Eşleşen kelimeler için video bulunamadı.")
        return
    
    cap_list = [cv2.VideoCapture(path) for path in video_paths]

    while True:
        for cap in cap_list:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                cv2.imshow("İşaret Dili Çeviri", frame)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            cap.release()

    cv2.destroyAllWindows()

# Ana Fonksiyon
def main():
    detected_sentence = recognize_speech()
    if detected_sentence:
        video_paths = find_video_files(detected_sentence)
        if video_paths:
            print(f"Oynatılacak Videolar: {video_paths}")
            play_videos(video_paths)  # Videoları sırayla oynat
        else:
            print("Cümledeki kelimeler için video bulunamadı.")
    else:
        print("Cümle algılanamadı.")

if __name__ == "main":
    main()